In [46]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import copy
import scipy

def f(lmd,c):
    # return scipy.special.expit(-lmd*c)
    y = 1 / (1 + np.exp(-lmd*c))
    return y

def h(err):
    y = 1 / (2 * err + 1)
    return y

def errorLp(p,data_pre,data_real):
    dist = np.linalg.norm(data_pre-data_real,ord=p)/(data_pre.shape[0]*data_pre.shape[1])
    return dist
  
#%%
#返回初始化权重参数   NxN 
def initialize_parameters_he(npop,N):
    parameters = np.zeros((npop,N*N))
    for i in range(npop):
        parameter = np.random.randn(N*(N-1)) * np.sqrt(2.0 / (N*(N-1)))
        lmd = np.random.uniform(0,1,N)
        #lmd = np.array([1]*N)
        parameters[i] = parameter.tolist()+lmd.tolist()
    return parameters
    

def rshape(X,N):
    temp = copy.copy(X)
    param = temp.reshape(N,N) 
    e = np.zeros((N,N))  
    lmd = param[N-1,:]
    for i in range(N):
        num = 0
        for j in range(N):
            if i is not j:
                e[i,j] = param[i,num] 
                num = num + 1
    return e,lmd
    
#jaya学习算法训练FCM参数
def jayaTrain(c_data,c_real,time,N,npop=4):
    X = initialize_parameters_he(npop,N)
#    for n in range(npop):
#        for i in range(N*N):
#            X[n,i] = min(X[:,i]) + np.random.random()*(max(X[:,i])-min(X[:,i]))
    fitness = np.zeros(npop)
    error = np.zeros(npop)
    worst= 0
    best = 0
    error_time = []
    c_pre = np.zeros((c_real.shape[0],c_real.shape[1]))
    for num in range(500):
        for n in range(npop):
            e,lmd = rshape(X[n,:],N)
            #print("c_data.shape: %s" % c_data.shape)
            c_pre = fcm(e,lmd,c_data,c_real,time)
            #print("c_pre, c_real: %s ,%s" % c_pre.shape, c_real.shape)
            fitness[n] = h(errorLp(2,c_pre,c_real)) 
            error[n] = errorLp(2,c_pre,c_real)
        worst = fitness.tolist().index(min(fitness))
        best = fitness.tolist().index(max(fitness))
        print(worst,best,fitness)
    
        Xx = np.zeros((npop,N*N))
        for n in range(npop):
            Xx[n,:] = X[n,:] + (X[best,:]-np.abs(X[n,:]))*np.random.random()-(X[worst,:]-np.abs(X[n,:]))*np.random.random()
            for i in range(N*N):
                if Xx[n,i] < min(Xx[:,i]):
                    Xx[n,i] = min(Xx[:,i])
                if Xx[n,i] > max(Xx[:,i]):
                    Xx[n,i] = max(Xx[:,i])
            e,lmd = rshape(Xx[n,:],N)
            c_pre = fcm(e,lmd,c_data,c_real,time)
            error2 = errorLp(2,c_pre,c_real)
            if error2 <= error[n]:
                X[n,:] = Xx[n,:]              
        error_time.append(error[best])
    e,lmd = rshape(X[best,:],N)
    return e,lmd,error_time

#模糊认知图递推过程      
def fcm(e,lmd,data_front,data,time):
    '''
    c_len = len(data)
    data_pre = np.zeros((time+1,c_len))
    data_pre[0] = data_front
    
    for t in range(0,time):
        for i in range(c_len):
            for j in range(c_len):
                if i is not j :
                    data_pre[t+1,i] += e[j,i]*data[t,j]
            data_pre[t+1,i] += data[t,i]
            data_pre[t+1,i] = f(lmd[i],data[t+1,i])
    
    '''
    c_len = len(data_front)
    data_pre = np.zeros((time,c_len))
    data_pre[0] = np.dot(data_front,e)
    for i in range(c_len):
        data[0,i] = f(lmd[i],data_pre[0,i])
    for t in range(0,time-1):
        data_pre[t+1] = np.dot(data[t],e)
        for i in range(c_len):
            data[t+1,i] = f(lmd[i],data_pre[t+1,i])
    return data_pre

#模糊认知图递推过程训练用   
def fcmm(e,lmd,data_front):
    c_len = len(data_front)
    data_pre = np.dot(data_front,e)
    for i in range(c_len):
        data_pre[i] = f(lmd[i],data_pre[i])
    
    return data_pre

In [47]:
#%%
def drawPre(title,preData,realData,dataNum=50):
    plt.title(title)
    plt.plot(range(dataNum), preData, color='green', label='predict');
    plt.plot(range(dataNum), realData, color='red', label='real');
    plt.legend();
    plt.xlabel('time');
    plt.ylabel('value');
    plt.show();        
    
def dataErr(data,label='none'):
    #输出图表
    plt.xlabel('time(day)')
    plt.ylabel('lost')
    plt.plot(data, label=label)
    plt.legend()
    plt.show()


In [ ]:
#%% 
if __name__ == "__main__":
    data = pd.read_csv('dataProcess.csv',index_col = 0)
    data = np.array(data)
    train_time = 504               #设置滑动窗口
    start = 1
    hour = 100   
    concept = data.shape[1]
    
    #data_train = data[240:600,:]   #20天做训练
    data_test = data[start+train_time:start+train_time+hour,:]     #3天做测试
    
    
    data_pre = np.zeros((data_test.shape[0],data_test.shape[1]))
    for i in range(0,hour):
        data_train = data[i+start:i+train_time+start,:]
        data_real = data[i+start+train_time-1,:]
        e,lmd,error_time = jayaTrain(data[i+start-1,:],data_train,train_time,concept,npop = 8)
        data_pre[i] = fcmm(e,lmd,data_real)

            
#    e,lmd = jayaTrain(data_train[0,:],data_train,240,6,npop = 8)
#    data_pre = fcm(e,lmd,data_test[0,:],50)
    

0 6 [0.98230668 0.98946702 0.98593798 0.98392917 0.98388021 0.98262131
 0.99001025 0.98715539]
0 6 [0.98300611 0.98946702 0.98593798 0.98392917 0.98388021 0.98428284
 0.99001025 0.98715539]
4 1 [0.98775585 0.99149363 0.98593798 0.98392917 0.98388021 0.9855214
 0.99001025 0.98728875]
3 4 [0.98775585 0.99149363 0.98593798 0.98392917 0.9951583  0.9855214
 0.99001025 0.98728875]
5 4 [0.98897009 0.99461589 0.99111017 0.9926416  0.9951583  0.9855214
 0.99001025 0.98728875]
7 4 [0.98897009 0.99461589 0.99111017 0.9926416  0.9951583  0.99352705
 0.99001025 0.98728875]
0 4 [0.98897009 0.99461589 0.99215199 0.9944542  0.99544111 0.99352705
 0.99001025 0.99133627]
7 4 [0.99425711 0.99461589 0.99215199 0.9944542  0.99544111 0.99352705
 0.99174589 0.99133627]
6 4 [0.99425711 0.99461589 0.99215199 0.99506184 0.99573914 0.9937787
 0.99174589 0.99204438]
7 4 [0.99486996 0.99461589 0.99215199 0.99506184 0.99573914 0.99482669
 0.99456237 0.99204438]
7 4 [0.99486996 0.99498526 0.99222024 0.99569132 0.995

2 4 [0.99882371 0.99875335 0.99869374 0.9988635  0.99893317 0.99886135
 0.9988485  0.99875134]
2 4 [0.99882371 0.99875335 0.99869374 0.9988635  0.99893317 0.99886135
 0.9988485  0.99875134]
7 4 [0.99882371 0.998795   0.99883831 0.9988635  0.99893317 0.99886135
 0.9988485  0.99875134]
7 4 [0.99882371 0.998795   0.99883831 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.998795   0.99884609 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.998795   0.99884609 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.998795   0.99884609 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.99880988 0.99884609 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.99880988 0.99884609 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.99880988 0.99884609 0.9988635  0.99893317 0.99886749
 0.99888305 0.99875134]
7 4 [0.99882371 0.99880988 0.99884609 0.9988635  0

2 6 [0.99900152 0.99897954 0.99893107 0.99896722 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99893107 0.99896722 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99893107 0.99896722 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99893107 0.99896722 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99893107 0.99896722 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99893107 0.99896722 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99896471 0.99901371 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99896471 0.99901371 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99896471 0.99901371 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99896471 0.99901371 0.9990038  0.99901127
 0.99902118 0.99897643]
2 6 [0.99900152 0.99897954 0.99896471 0.99901371 0

7 6 [0.99900152 0.99902352 0.99900516 0.99901371 0.9990038  0.99901844
 0.99902453 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901371 0.9990038  0.99901844
 0.99902453 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901371 0.9990038  0.99901844
 0.99902453 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901371 0.9990038  0.99901844
 0.99902453 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901371 0.9990038  0.99901844
 0.99902845 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901371 0.9990038  0.99901844
 0.99902845 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901515 0.9990038  0.99901844
 0.99902845 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901515 0.9990038  0.99901844
 0.99902845 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901515 0.9990038  0.99901844
 0.99902845 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901515 0.9990038  0.99902785
 0.99902845 0.99900144]
7 6 [0.99900152 0.99902352 0.99900516 0.99901515 0

3 5 [0.99903773 0.99903027 0.9990444  0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 5 [0.99903773 0.99903027 0.9990444  0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 5 [0.99903773 0.99903027 0.9990444  0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99903773 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99903773 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99903773 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99903773 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99903773 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99903773 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99904045 0.99903027 0.99905237 0.99901515 0.99904216 0.99904632
 0.99903011 0.99903901]
3 2 [0.99904045 0.99903027 0.99905237 0.99902371 0

...

6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0.9998129  0.99981875
 0.99979652 0.99980406]
6 1 [0.99981789 0.99983426 0.99980647 0.99982653 0

6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980406]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980406]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980406]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980522]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980522]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980522]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980522]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980522]
6 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99979953 0.99980522]
7 1 [0.99981789 0.99983465 0.99981736 0.99982653 0.99981769 0.99982043
 0.99983008 0.99980522]
7 1 [0.99981789 0.99983465 0.99981736 0.99982653 0

6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.999485   0.99950902]
6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.999485   0.99950902]
6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99950515 0.99950902]
6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99950515 0.99950902]
6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99950515 0.99950902]
6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99950515 0.99950902]
6 1 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99950515 0.99950902]
7 6 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99958145 0.99950902]
7 6 [0.99951576 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99958145 0.99950902]
7 6 [0.99953504 0.99956014 0.9995459  0.99951086 0.99955345 0.99951781
 0.99958145 0.99950902]
3 6 [0.99953504 0.99956014 0.9995459  0.99951086 0

1 6 [0.99960869 0.99959945 0.99961052 0.99960817 0.99960565 0.99960949
 0.99961279 0.99961001]
1 6 [0.99960869 0.99960355 0.99961052 0.99960817 0.99960565 0.99960949
 0.99961279 0.99961001]
1 6 [0.99960869 0.99960355 0.99961052 0.99960817 0.99960565 0.99961054
 0.99961279 0.99961001]
1 6 [0.99960869 0.99960355 0.99961052 0.99960817 0.99960565 0.99961054
 0.99961279 0.99961088]
1 6 [0.99960869 0.99960355 0.99961052 0.99960817 0.99960565 0.99961054
 0.99961279 0.99961088]
1 6 [0.99960869 0.99960355 0.99961052 0.99960817 0.99960565 0.99961054
 0.99961279 0.99961088]
1 6 [0.99960869 0.99960355 0.99961052 0.99960817 0.99960565 0.99961054
 0.99961279 0.99961088]
1 4 [0.99960869 0.99960355 0.99961052 0.99960817 0.99961375 0.99961054
 0.99961279 0.99961088]
3 4 [0.99960869 0.99961261 0.99961052 0.99960817 0.99961375 0.99961054
 0.99961279 0.99961088]
3 4 [0.99960869 0.99961261 0.99961052 0.99960817 0.99961375 0.99961054
 0.99961279 0.99961088]
3 4 [0.99960869 0.99961261 0.99961052 0.99960817 0

6 5 [0.99961902 0.9996195  0.99961961 0.9996175  0.99962141 0.99962293
 0.99961461 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.9996175  0.99962141 0.99962293
 0.99961461 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.9996175  0.99962152 0.99962293
 0.99961461 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961461 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961822 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961822 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961822 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961822 0.99962156]
6 5 [0.99961902 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961822 0.99962156]
6 5 [0.99961966 0.9996195  0.99961961 0.99962129 0.99962152 0.99962293
 0.99961822 0.99962156]
6 5 [0.99961966 0.9996195  0.99961961 0.99962129 0

In [43]:
print(errorLp(2,data_pre,data_test))
drawPre("CO",data_pre[:,0],data_test[:,0],dataNum = data_test.shape[0])
drawPre("NO2",data_pre[:,1],data_test[:,1],dataNum = data_test.shape[0])
drawPre("SO2",data_pre[:,2],data_test[:,2],dataNum = data_test.shape[0])
drawPre("O3",data_pre[:,3],data_test[:,3],dataNum = data_test.shape[0])
drawPre("PM25",data_pre[:,4],data_test[:,4],dataNum = data_test.shape[0])
drawPre("PM10",data_pre[:,5],data_test[:,5],dataNum = data_test.shape[0])

TypeError: '(slice(505, 605, None), slice(None, None, None))' is an invalid key

In [37]:
data[0:5]

array([[0.05885026, 0.0425228 , 0.31421085, 0.06780559, 0.0786056 ,
        0.09601975],
       [0.51476713, 0.49607794, 0.54681457, 0.55158903, 0.49282915,
        0.42775458],
       [0.5266764 , 0.51182872, 0.55151761, 0.51767574, 0.55064135,
        0.54117328],
       [0.50121232, 0.50292313, 0.49391819, 0.48316074, 0.49161422,
        0.50003013],
       [0.41733001, 0.49898507, 0.54563713, 0.45277743, 0.48399093,
        0.50632553]])

In [38]:
data[5]

array([0.50136892, 0.48729358, 0.54335933, 0.50083837, 0.46449971,
       0.50022258])